In [1]:
import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr

from sklearn.linear_model import LinearRegression
from tqdm import tqdm
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
# from pandas import datetime
from pmdarima.arima import auto_arima
import statsmodels.api as sm

In [2]:
path = './open'
list_name = 'Stock_List.csv'
sample_name = 'sample_submission.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list


,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [3]:
#start_date = '20210104'
start_date = '20210328'
end_date = '20211031'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days.head())
display(Business_days.tail())


WEEKDAY of "start_date" : 6
NUM of WEEKS to "end_date" : 43
HOW MANY "Business_days" : (155, 1)


,Date
0,2021-03-29
1,2021-03-30
2,2021-03-31
3,2021-04-01
4,2021-04-02


,Date
150,2021-10-25
151,2021-10-26
152,2021-10-27
153,2021-10-28
154,2021-10-29


In [4]:
# model = LinearRegression()
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date).reset_index()
    data.Close = data.Close.ffill()
    data.index = data["Date"]
    data.drop('Date', axis=1, inplace=True)
    
#     from statsmodels.tsa.stattools import adfuller
#     result = adfuller(data.Close)
#     print(result[0])
#     print("p-value", result[1])

#     plt.plot(data.Close)
#     plt.show()

#     plt.plot(data.Close.shift())
#     plt.show()
    
#     close_diff = data.Close - data.Close.shift()
#     plt.plot(close_diff)
#     plt.show()
    
    
#     result = adfuller(close_diff[1:])
#     print(result[0])
#     print(result[1])
    
#     import statsmodels.api as sm
    
#     fig = plt.figure(figsize=(20, 8))
#     ax1 = fig.add_subplot(212)
#     fig = sm.graphics.tsa.plot_acf(close_diff[1:], ax=ax1)
#     plt.show()

    
#     # autoarima
    arima_data = data[:-1]
#     model = auto_arima(arima_data.Close, trace=True)
#     model.fit(arima_data.Close)
#     forecast = model.predict(n_periods=5)
#     predictions = list(forecast)
    
#     ARIMA(p, d, q)
#     d=0 : ARMA(p, q), 정상성 만족
#     p=0 : IMA(d,q), d번 차분하면 MA(q)모형을 따르게 됨
#     q=0 : ARI(p,d), d번 차분한 시계열이 AR(p)모형을 따르게 됨

    model = ARIMA(arima_data.Close, order=(0, 1, 0))
    model_fit = model.fit()
    
#     start_index = datetime(2021, 11, 1)
#     end_index = datetime(2021, 11, 5)
#     forecast = model_fit.forecast(steps=5)
    
    
#     predictions = list(forecast)
#     sample_submission.loc[:,code] = predictions * 2
# sample_submission.isna().sum().sum()

  0%|          | 0/370 [00:00<?, ?it/s]C:\Users\user\.conda\envs\stockpredict\lib\site-packages\statsmodels\tsa\base\tsa_model.py:593: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Users\user\.conda\envs\stockpredict\lib\site-packages\statsmodels\tsa\base\tsa_model.py:593: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\Users\user\.conda\envs\stockpredict\lib\site-packages\statsmodels\tsa\base\tsa_model.py:593: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
  0%|          | 1/370 [00:00<03:12,  1.92it/s]C:\Users\user\.co

KeyboardInterrupt: 

In [ ]:
columns = list(sample_submission.columns[1:])

columns = ['Day'] + [str(x).zfill(6) for x in columns]

sample_submission.columns = columns

In [ ]:
sample_submission.to_csv('arima.csv',index=False)

In [ ]:
sample_submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,29086.680778,36100.0,61800.0,148783.151263,96400.0,17370.101623,48300.0,83514.169160,98756.952769,...,45700.0,50600.0,82259.219858,34448.444282,26600.0,50813.475177,325000.0,255000.0,27700.0,17700.0
1,2021-11-02,29235.633255,36100.0,61800.0,148706.739806,96400.0,17392.306611,48300.0,83615.760482,98366.041434,...,45700.0,50600.0,82518.439716,34400.408338,26600.0,50626.950355,325000.0,255000.0,27700.0,17700.0
2,2021-11-03,29465.358858,36100.0,61800.0,148824.204101,96400.0,17399.085585,48300.0,83706.159646,98379.929095,...,45700.0,50600.0,82777.659574,34355.651868,26600.0,50440.425532,325000.0,255000.0,27700.0,17700.0
3,2021-11-04,29621.771866,36100.0,61800.0,149041.996951,96400.0,17385.397974,48300.0,83786.599675,98381.531229,...,45700.0,50600.0,83036.879433,34313.950978,26600.0,50253.900709,325000.0,255000.0,27700.0,17700.0
4,2021-11-05,29636.853208,36100.0,61800.0,149193.879138,96400.0,17362.420265,48300.0,83858.177752,98025.599508,...,45700.0,50600.0,83296.099291,34275.097059,26600.0,50067.375887,325000.0,255000.0,27700.0,17700.0
5,2021-11-29,29086.680778,36100.0,61800.0,148783.151263,96400.0,17370.101623,48300.0,83514.169160,98756.952769,...,45700.0,50600.0,82259.219858,34448.444282,26600.0,50813.475177,325000.0,255000.0,27700.0,17700.0
6,2021-11-30,29235.633255,36100.0,61800.0,148706.739806,96400.0,17392.306611,48300.0,83615.760482,98366.041434,...,45700.0,50600.0,82518.439716,34400.408338,26600.0,50626.950355,325000.0,255000.0,27700.0,17700.0
7,2021-12-01,29465.358858,36100.0,61800.0,148824.204101,96400.0,17399.085585,48300.0,83706.159646,98379.929095,...,45700.0,50600.0,82777.659574,34355.651868,26600.0,50440.425532,325000.0,255000.0,27700.0,17700.0
8,2021-12-02,29621.771866,36100.0,61800.0,149041.996951,96400.0,17385.397974,48300.0,83786.599675,98381.531229,...,45700.0,50600.0,83036.879433,34313.950978,26600.0,50253.900709,325000.0,255000.0,27700.0,17700.0
9,2021-12-03,29636.853208,36100.0,61800.0,149193.879138,96400.0,17362.420265,48300.0,83858.177752,98025.599508,...,45700.0,50600.0,83296.099291,34275.097059,26600.0,50067.375887,325000.0,255000.0,27700.0,17700.0


In [ ]:

start_date = '20211101'
end_date = '20211105'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )


real=[]
real_data=pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\stock_predict2\\open\\sample_submission.csv')
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()
    data.Close = data.Close.bfill()

    real_data_close = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    real_data_close_flatten = np.ravel(real_data_close.to_numpy())
    # print(real_data_close_flatten)
    

    real_data.loc[:4,code]=real_data_close_flatten


type(real_data)
type(sample_submission)

label_np = real_data.to_numpy()[0:5,1:]
pred_np = sample_submission.to_numpy()[0:5, 1:]
print(pred_np)


result=np.mean(abs(label_np-pred_np)/label_np)*100
print(result)

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 44
HOW MANY "Business_days" : (5, 1)


100%|██████████| 370/370 [01:23<00:00,  4.42it/s]

[[29086.680778493344 36100.0 61800.0 ... 255000.0 27700.0 17700.0]
 [29235.63325486544 36100.0 61800.0 ... 255000.0 27700.0 17700.0]
 [29465.358857505194 36100.0 61800.0 ... 255000.0 27700.0 17700.0]
 [29621.771866186642 36100.0 61800.0 ... 255000.0 27700.0 17700.0]
 [29636.853207753 36100.0 61800.0 ... 255000.0 27700.0 17700.0]]
4.8891437316242135
